In [32]:
import replicate
# from dotenv import load_dotenv
# import os
# load_dotenv()
# os.environ.get('REPLICATE_API_TOKEN')


#### Replicate API checking for LLMA 2

In [33]:
# Online Model 
output = replicate.run(
  "meta/llama-2-7b:acdbe5a4987a29261ba7d7d4195ad4fa6b62ce27b034f989fcb9ab0421408a7c",
  input={
    "debug": False,
    "top_k": 250,
    "top_p": 0.95,
    "prompt": "What is Java?",
    "temperature": 0.95,
    "max_new_tokens": 500,
    "min_new_tokens": -1,
    "repetition_penalty": 1.15,
    "repetition_penalty_sustain": 256,
    "token_repetition_penalty_decay": 128
  }
)
print(output)

<generator object Prediction.output_iterator at 0x000001F54BC93AC0>


In [34]:
result = list(output)
generated_text = "".join(result)

print(generated_text)

 Why did Java become so famous in the past few years, you know with a lot of companies taking and use it as their language for writing applications on the back end that you’re kind of making a comparison here between server side development. everyone is talking about, they’ve been using PHP or Ruby but now they want to do this thing called Java. So why would I want to learn this language? Well let me tell you actually that there are four reasons especially today really why you may wanna consider learning more about Java. And we already covered one of them which is job market opportunities right at any given time if you go around asking like oh hey I need somebody who understands Java on my team what happens when people aren’t going well I don’t mind hiring someone I just I don’t know that doesn’t sound familiar to what else might happen right that person said well hold on wait how am I gonna vouch or even promote your skills out there amongst others being a programmer these days when y

# Online model checking

<!-- Online Model checking -->

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone




c:\Users\anoop\anaconda3\envs\mchatbot\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [4]:
# This one is working fine 


from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Replicate

llm = Replicate(
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model="meta/llama-2-7b:acdbe5a4987a29261ba7d7d4195ad4fa6b62ce27b034f989fcb9ab0421408a7c",
    input={"temperature": 0.75, "max_length": 500, "top_p": 1}
)

streaming was transfered to model_kwargs.
                    Please confirm that streaming is what you intended.


In [5]:
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV')

In [6]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [12]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()


In [14]:
#Initializing the Pinecone
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="statistics-chatbot"
docsearch=Pinecone.from_existing_index(index_name, embeddings)

In [26]:
index_name="statistics-chatbot"

docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What is Statistics?"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(page_content='Page 6 \n \nSH                                                       Statistics for Data Science  \n \nStatistics: \nStatistics is a branch of mathematics that deals with the study of collecting, \nanalysing, interpreting, presenting, and organizing data in a particular manner. Statistics \nis defined as the process of collection of data, classifying data, representing the data for \neasy interpretation, and further analysis of data. Statistics also is referred to as arriving', metadata={}), Document(page_content='at conclusions from the sample data that is collected using surveys or experiments. \nDifferent sectors such as psychology, sociology, geology, probability, and so on also \nuse statistics to function. \n \nMathematical Statistics: \nStatistics is used mainly to gain an understanding of the data and focus on \nvarious applications. Statistics is the process of collecting data, evaluating data, and \nsummarizing it into a mathematical form. Initi

In [27]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""


In [28]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [29]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)


In [30]:
user_input=input(f"Input Prompt:")
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :  
Hey there!
I have almost everything that I need from your side.

Firstly what a good predictor does? A good predictor takes only those features into consideration that are most important in determining outcome variable (to be predicted). It doesn’t take irrelevant (unimportant) attributes.

Meanwhile, we can either use both the classification and regression trees together but it will lead us towards overfitting so if you want we can do this after building a strong classifier-regression tree first via the whole dataset.
So basically, we can go for 2 ways:
(i) Try using random forest to see how well(higher accuracy) it performs in prediction as compared to decision tree itself.
(ii) Choose the small subset of attributes via splitting ratio from those features that has high variance and then build reg tree with those features.

However, like I mentioned above selection depends upon the importance of each feature to determine the value of prediction/outcome variable. You shoul

In [24]:
result=qa({"query": "What is stats?"})
print("Response : ", result["result"])

Response :  Statistics is the process of collecting data, analysing data, and summarising it into a mathematical form. Mathematics plays a vital role even today in different domains like psychology, sociology, biostatistics etc., where statistical methods are adopted widely to analyse data. Here also the basic purpose behind this adoption was to arrive at some conclusion by drawing upon a sample data which has been collected through either survey techniques or experimentation.

Other answers (if any)

